In [2]:
import os
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Step 1: Load images from "yes" and "no" folders
yes_folder = "C:/Image Processing/Project/P2/yes"  # Specify path to folder containing tumor images
no_folder = "C:/Image Processing/Project/P2/no"   # Specify path to folder containing non-tumor images

# Function to load images from folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

# Load tumor images
tumor_images = load_images_from_folder(yes_folder)

# Load non-tumor images
non_tumor_images = load_images_from_folder(no_folder)

# Step 2: Preprocess images
# Convert images to grayscale and resize
def preprocess_images(images):
    processed_images = []
    for img in images:
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        resized_img = cv2.resize(gray_img, (100, 100))  # Resize to a fixed size for consistency
        processed_images.append(resized_img)
    return processed_images

tumor_images_processed = preprocess_images(tumor_images)
non_tumor_images_processed = preprocess_images(non_tumor_images)

# Step 3: Extract features
# Flatten images and concatenate tumor and non-tumor features
tumor_features = np.array([img.flatten() for img in tumor_images_processed])
non_tumor_features = np.array([img.flatten() for img in non_tumor_images_processed])
X = np.concatenate((tumor_features, non_tumor_features))

# Create labels
y = np.concatenate((np.ones(len(tumor_features)), np.zeros(len(non_tumor_features))))

# Step 4: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Train a classifier
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Step 6: Evaluate classifier's performance
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Step 7: Predict tumor presence in new images
# Load a new image and preprocess it
new_image_path = "C:/Image Processing/Project/P2/no/27 no.jpg"  # Specify path to new image using forward slashes
new_img = cv2.imread(new_image_path)

# Check if the image is loaded successfully
if new_img is None:
    print("Error: Unable to load the image from the specified path.")
else:
    # Preprocess the image and predict
    new_img_gray = cv2.cvtColor(new_img, cv2.COLOR_BGR2GRAY)
    new_img_resized = cv2.resize(new_img_gray, (100, 100))
    new_img_processed = new_img_resized.flatten()

    # Predict using the trained model
    prediction = svm_model.predict([new_img_processed])[0]
    if prediction == 1:
        print("Tumor detected!")
    else:
        print("No tumor detected.")


Accuracy: 0.7647058823529411
Tumor detected!
